In [1]:
# Install required packages
%pip install chromadb
%pip install llama-index
%pip install pypdf
%pip install llama-index chromadb --quiet
%pip install sentence-transformers
%pip install pydantic==1.10.11

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from langchain.llms import LlamaCpp
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext, LLMPredictor
from llama_index.vector_stores import ChromaVectorStore
from llama_index.storage.storage_context import StorageContext
from llama_index.embeddings import HuggingFaceEmbedding
from IPython.display import Markdown, display
import chromadb

'''
llm = LlamaCpp(model_path="../llms/llama-2-13b-chat.Q4_K_M.gguf",
        n_gpu_layers=-1,
        n_batch=512,
        max_tokens=512,
        n_ctx=2048,
        f16_kv=True,
        verbose=False,
        temperature=0.1,
)
'''

llm = LlamaCpp(model_path="../llms/llama-2-13b-chat.Q4_K_M.gguf",
               
    # Anzahl der GPU-Layer, die verwendet werden sollen. 
    # -1 bedeutet, dass alle verfügbaren Layer verwendet werden.
    n_gpu_layers=-1,

    # Anzahl der Anfragen, die gleichzeitig verarbeitet werden können (typischer Bereich: 1-1024).
    n_batch=512,

    # Maximale Anzahl von Tokens pro Anfrage (typischer Bereich: 128-1024).
    max_tokens=512,

    # Die Größe des Kontexts (in Tokens), die das Modell berücksichtigt (typischer Bereich: 1024-4096).
    n_ctx=4096,

    # Verwendung von 16-Bit-Floats für Schlüssel/Werte im Transformer (true/false).
    # Das Aktivieren kann die Effizienz erhöhen und den Speicherbedarf verringern, kann aber die Genauigkeit leicht beeinträchtigen.
    f16_kv=False,

    # Steuert die Ausgabe von Debug-Informationen (true/false).
    verbose=False,

    # Temperatur für die Zufälligkeit bei der Token-Auswahl (typischer Bereich: 0.1-1.0).
    # Niedrigere Werte machen die Vorhersagen konsistenter.
    temperature=0.0,

    # Vermeidung von Wiederholungen in den Antworten (typischer Bereich: 1.0-2.0).
    # Höhere Werte erhöhen die Diversität, senkt aber die relevanz des Inhalts.
    repetition_penalty=1.1,

    # Maximale Anzahl von Generierungsschritten (typischer Bereich: 50-500).
    #Ein höherer Wert erlaubt längere Antworten, benötigt aber mehr Rechenzeit.
    max_steps=250,

    # Wahrscheinlichkeit für die Auswahl der nächsten Tokens (typischer Bereich: 0.5-1.0).
    # Ein niedrigerer Wert führt zu vorhersehbareren Antworten, während ein höherer Wert die Diversität der Antworten erhöht.
    top_p=0.25,

    # Begrenzt die Auswahlmöglichkeiten für das nächste Token (typischer Bereich: 10-50).
    # top_k=40,

    # Parameter für Lernprozess:
               
    # Lernrate für das Training des Modells (typischer Bereich: 1e-5 bis 1e-3).
    # learning_rate=1e-4,

    # Größe der Mini-Batches während des Trainings (typischer Bereich: 8-64).
    # train_batch_size=32,

    # Anzahl der Epochen während des Trainings (typischer Bereich: 1-10).
    # epochs=3,
)


llm_predictor = LLMPredictor(llm=llm)
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, embed_model='local')    

/home/tpllmws23/environments/llm/lib/python3.10/site-packages/langchain/utils/utils.py:159: UserWarning: WARNING! repetition_penalty is not default parameter.
                repetition_penalty was transferred to model_kwargs.
                Please confirm that repetition_penalty is what you intended.
  warnings.warn(
/home/tpllmws23/environments/llm/lib/python3.10/site-packages/langchain/utils/utils.py:159: UserWarning: WARNING! max_steps is not default parameter.
                max_steps was transferred to model_kwargs.
                Please confirm that max_steps is what you intended.
  warnings.warn(
ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:
  Device 0: NVIDIA TITAN RTX, compute capability 7.5
llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from ../llms/llama-2-13b-chat.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: - tensor    0:                token_embd.weigh

In [3]:
db = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db.get_or_create_collection("pruefungsamt")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

if not os.path.exists('./chroma_db/'):
    documents = SimpleDirectoryReader("./main_data/").load_data()
    index = VectorStoreIndex.from_documents(
        documents,
        storage_context=storage_context,
        service_context=service_context
    )
    
index = VectorStoreIndex.from_vector_store(
    vector_store,
    service_context=service_context,
)

In [3]:
def printA(question):
    print('A: ' + question)

def printQ(question):
    print('Q: ' + question)

def generate_initial_message():
    message = '''
    Welcome to the Q&A Chatbot!
    Please phrase your questions clearly and specifically, and focus on one topic per question for more precise answers.
    '''
    return message

printA(generate_initial_message())

def generate_system_prompt():
    style = '''
    1. Maintain neutrality in the response.
        1.1. Include precise and specific details.
        1.2. Ensure complete information is provided regarding the query.
        1.3. Keep the response objective.
        1.4. Use clear, straightforward language.
        1.5. Avoid redundancy and irrelevant information.
        1.6. Use bullet lists if it makes sense to display multiple data.
    '''

    listStructure = '''
    Structure bulleted lists as follows:
    1) Topic
        1.1) Subtopic
            1.1.1) Subsubtopic
            ....
    '''

    validation = '''
    1. Before processing the question, validate that it strictly pertains to information related to the university, studies, or exams. Do not answer questions unrelated to these topics.
        1.1. Ensure that the response is contextually aligned with any previous queries or the overall conversation flow.
    '''
    
    user_guidance = '''
    1. Please phrase your questions clearly and specifically.
        1.1. Focus on one topic per question for more precise answers.
    '''

    privacy_notes = '''
    1. Do not share personal data in your questions.
        1.1. All queries are processed anonymously to protect your privacy.
    '''

    important_information = '''
    1. Distinguish clearly between master's and bachelor's programs.
        1.1. If the question is ambiguous regarding the program level (master's or bachelor's), ask for clarification.
        1.2. Always write in brackets whether it is a "Master's" or "Bachelor's" degree program.
    2. Ensure clarity on which degree program the query is about.
        2.1. Ask for specifics if necessary.
    '''

    header = '''
    Provide a very brief summary of the question.
    '''

    body = '''
    Directly answer the question, then provide detailed explanations if necessary.
    '''

    footer = '''
    1. Remind the user to adhere to privacy notes in case they enter personal information.
        1.1. Refer to the User Guidance if the questions are not asked precisely enough.
        1.2. Cite the source of the information in the documents.
    '''

    system_prompt = f'''
    // Instructions for the LLM:
    // Please follow these style guidelines when answering the question:
    {style}
    {listStructure}

    // Validate the relevance of the question:
    {validation}
    
    // User Guidance: 
    {user_guidance}
    
    // Privacy Notes:
    {privacy_notes}

    // Important considerations before answering:
    {important_information}

    // Structure the response as follows:
    
    //1. Header:
    {header}
    
    //2. Body:
    {body}
    
    //3. Footer:
    {footer}
    '''

    return system_prompt

def generate_system_prompt_withQuestion(question):
    system_prompt_withQuestion = f'''
    {generate_system_prompt()}
    // Question:
    {question}
    '''
    return system_prompt_withQuestion

A: 
    Welcome to the Q&A Chatbot!
    Please phrase your questions clearly and specifically, and focus on one topic per question for more precise answers.
    


In [9]:
question = "Write a sarcastic commentry about the phrase: Thank you for the Coffee"

query_engine = index.as_query_engine(streaming=True, similarity_top_k=5)
response_stream = query_engine.query(question)
response_stream.print_response_stream()


Please let me know if this is what you're looking for.

In [6]:
question = "What are the requirements for studying applied computer science at the HTWG?"

In [28]:
question = "How many ects do i need for the master degree at the HTWG?"

In [7]:
print(question)
prepared_question = generate_system_prompt_with_Question(question)
print(prepared_question)
query_engine = index.as_query_engine()
response = query_engine.query(prepared_question)
display(Markdown(f"<b>{response}</b>"))

Write a sarcastic commentry about the phrase: Thank you for the Coffee


NameError: name 'generate_system_prompt_with_Question' is not defined

In [36]:
question = "What are the requirements for studying informatics in masters degree at the HTWG?. The sources should be cited in the style: '[number]'"
query_engine = index.as_query_engine()
response = query_engine.query(question)
display(Markdown(f"<b>{response}</b>"))



<b>

Please provide the answer with proper formatting and explanation.

Note: Please note that I have provided the context information as a text file, but if you need it in any other format, please let me know.</b>

In [8]:
# stream query
query_engine = index.as_query_engine(streaming=True, similarity_top_k=5)
response_stream = query_engine.query(question)
response_stream.print_response_stream()

AttributeError: 'Response' object has no attribute 'print_response_stream'

In [1]:
from llama_index.query_engine import CitationQueryEngine

question = "What are the requirements for studying MSI in masters degree at the HTWG? The sources should be cited in the style: '[number]'"

query_engine = CitationQueryEngine.from_args(index, service_context=service_context, similarity_top_k=5, citation_chunk_size=1024)
response = query_engine.query(question)
display(Markdown(f"<b>{response}</b>"))

NameError: name 'index' is not defined

In [6]:
print(len(response.source_nodes))

for node in response.source_nodes:
    print(node.node.metadata)
    print(node.node.get_text())


0
